# 🎵 Machine Listening Workshop - Demo

Bem-vindo ao workshop de **Machine Listening**! Este notebook demonstra processamento de áudio e análise usando Python.

## 📦 Setup do Ambiente

Primeiro, vamos configurar o ambiente automaticamente.

In [ ]:
# Detecta se está no Colab e instala dependências
import sys
import subprocess

try:
    import google.colab
    IN_COLAB = True
    print("🌐 Rodando no Google Colab")
    
    # URL do repositório
    REPO_URL = "https://raw.githubusercontent.com/zepadovani/2025_FU_workshop/main"
    REQUIREMENTS_URL = f"{REPO_URL}/requirements.txt"
    
    print("\n📦 Instalando dependências...")
    
    # Tenta baixar e instalar do requirements.txt
    try:
        subprocess.run(["wget", "-q", REQUIREMENTS_URL, "-O", "requirements.txt"], check=True)
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"], check=True)
        print("✅ Dependências instaladas do requirements.txt")
    except:
        # Fallback: instalação manual
        print("⚠️  Instalando pacotes manualmente...")
        packages = ["librosa", "soundfile", "gradio", "matplotlib"]
        for pkg in packages:
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=True)
            print(f"  ✓ {pkg}")
        print("✅ Instalação completa")
    
except ImportError:
    IN_COLAB = False
    print("💻 Rodando localmente (usando ambiente Pixi)")

## 📚 Importar Bibliotecas

Agora vamos importar as bibliotecas necessárias.

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import gradio as gr
from IPython.display import Audio, display

print("✅ Bibliotecas importadas com sucesso!")
print(f"Librosa version: {librosa.__version__}")
print(f"Gradio version: {gr.__version__}")

## 🎼 Gerar Áudio Sintético

Vamos criar um áudio de exemplo para demonstração.

In [ ]:
# Parâmetros
sample_rate = 22050
duration = 2.0  # segundos
frequency = 440  # Hz (nota Lá)

# Gera um tom senoidal
t = np.linspace(0, duration, int(sample_rate * duration))
audio_signal = 0.5 * np.sin(2 * np.pi * frequency * t)

# Adiciona um fade in/out
fade_samples = int(0.1 * sample_rate)
audio_signal[:fade_samples] *= np.linspace(0, 1, fade_samples)
audio_signal[-fade_samples:] *= np.linspace(1, 0, fade_samples)

print(f"✅ Áudio gerado: {duration}s @ {sample_rate} Hz")
print(f"   Frequência: {frequency} Hz")
print(f"   Shape: {audio_signal.shape}")

# Reproduz o áudio
display(Audio(audio_signal, rate=sample_rate))

## 📊 Análise Espectral

Vamos visualizar o espectrograma do áudio.

In [ ]:
# Calcula a STFT (Short-Time Fourier Transform)
D = librosa.stft(audio_signal)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

# Visualiza o espectrograma
plt.figure(figsize=(12, 4))
librosa.display.specshow(S_db, x_axis="time", y_axis="hz", sr=sample_rate, cmap="viridis")
plt.colorbar(format="%+2.0f dB")
plt.title("Espectrograma")
plt.tight_layout()
plt.show()

print(f"Shape do espectrograma: {S_db.shape}")

## 🎯 Análise de Características

Vamos extrair características do áudio usando Librosa.

In [ ]:
# Duração
duration_calc = librosa.get_duration(y=audio_signal, sr=sample_rate)

# Tempo (BPM) - para um tom puro, pode não ser muito significativo
tempo, beats = librosa.beat.beat_track(y=audio_signal, sr=sample_rate)

# Zero Crossing Rate
zcr = librosa.feature.zero_crossing_rate(audio_signal)

# Spectral Centroid
spectral_centroids = librosa.feature.spectral_centroid(y=audio_signal, sr=sample_rate)

# MFCCs
mfccs = librosa.feature.mfcc(y=audio_signal, sr=sample_rate, n_mfcc=13)

print("📊 Características Extraídas:")
print(f"  Duração: {duration_calc:.2f}s")
print(f"  Tempo: {tempo:.2f} BPM")
print(f"  Zero Crossing Rate (média): {zcr.mean():.4f}")
print(f"  Spectral Centroid (média): {spectral_centroids.mean():.2f} Hz")
print(f"  MFCCs shape: {mfccs.shape}")

## 🎨 Interface Gradio Interativa

Vamos criar uma interface web interativa com Gradio!

In [ ]:
def analyze_audio(audio_input):
    """
    Analisa um arquivo de áudio e retorna informações + espectrograma.
    """
    try:
        # Processa o input
        if isinstance(audio_input, tuple):
            sr, y = audio_input
            # Converte para mono se necessário
            if len(y.shape) > 1:
                y = np.mean(y, axis=1)
            # Normaliza
            y = y.astype(np.float32)
            if y.max() > 1.0:
                y = y / np.iinfo(np.int16).max
        else:
            y, sr = librosa.load(audio_input, sr=None)
        
        # Análise
        duration = librosa.get_duration(y=y, sr=sr)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y).mean()
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
        
        # Espectrograma
        D = librosa.stft(y)
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        
        fig, ax = plt.subplots(figsize=(10, 4))
        librosa.display.specshow(S_db, x_axis="time", y_axis="hz", sr=sr, ax=ax, cmap="viridis")
        fig.colorbar(ax.collections[0], ax=ax, format="%+2.0f dB")
        ax.set_title("Espectrograma")
        plt.tight_layout()
        
        # Resultado textual
        result = f"""
### ✅ Análise Completa!

**📊 Informações:**
- Duração: {duration:.2f}s
- Taxa de Amostragem: {sr} Hz
- Tempo Estimado: {tempo:.2f} BPM
- Zero Crossing Rate: {zcr:.4f}
- Spectral Centroid: {centroid:.2f} Hz
        """
        
        return result, fig
        
    except Exception as e:
        return f"❌ Erro: {str(e)}", None


# Cria a interface
with gr.Blocks(title="Machine Listening Demo") as demo:
    gr.Markdown("""
    # 🎵 Machine Listening - Demo Interativa
    Faça upload de um arquivo de áudio ou grave usando o microfone!
    """)
    
    with gr.Row():
        audio_input = gr.Audio(label="Áudio", type="filepath", sources=["upload", "microphone"])
    
    analyze_btn = gr.Button("🔍 Analisar", variant="primary")
    
    text_output = gr.Markdown()
    plot_output = gr.Plot()
    
    analyze_btn.click(
        fn=analyze_audio,
        inputs=audio_input,
        outputs=[text_output, plot_output]
    )

# Lança a interface (share=True para link público no Colab)
demo.launch(share=IN_COLAB)

## 🎓 Conclusão

Parabéns! Você completou o workshop básico de Machine Listening.

### 📚 Próximos Passos:

1. Experimente com seus próprios arquivos de áudio
2. Explore outras features do Librosa ([documentação](https://librosa.org/))
3. Implemente classificação de áudio com ML
4. Crie visualizações mais avançadas

### 🔗 Recursos:

- [Librosa Documentation](https://librosa.org/doc/latest/index.html)
- [Gradio Documentation](https://www.gradio.app/docs/)
- [Audio Signal Processing for ML](https://www.youtube.com/watch?v=iCwMQJnKk2c)

---

**Happy Learning! 🎧🎶**